In [11]:
import os
import gc
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import scipy.stats  as ss

from statsmodels.api import ProbPlot


warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_rows', None)

In [12]:
train_df = pd.read_csv("./Cleaned_train.csv")
test_df = pd.read_csv("./Cleaned_test.csv")

In [13]:
# 类别、离散特征
cate_col_list = ['ProductCD','P_emaildomain','R_emaildomain','DeviceType','DeviceInfo','addr1','addr2']

#循环加入特征
for i in range(1,7):
    cate_col_list.append('card' + str(i))
for i in range(1 ,10):
    cate_col_list.append('M' + str(i))
for i in range(12 ,39):
    cate_col_list.append('id_' + str(i // 10) + str(i % 10))
#连续型特征
num_col_list = [x for x in train_df.columns if x not in cate_col_list]
num_col_list.remove('isFraud')

In [14]:
object_cate_list = [x for x in cate_col_list if train_df[x].dtype == 'object']

# Feature Engineering

- 目的并不是要做这个项目到极致，因为采用的是集成学习这种复杂模型，暂时不必采用分箱处理
- 采用树模型，暂时也不必做数据缩放处理

## Feature Encoding

对object数据类型的类别特征做mean encoding

In [17]:
%run Featrue_Engineering_utils.ipynb

In [18]:
meanencoder = MeanEncoder(categorical_features= object_cate_list)

In [19]:
temp_df = meanencoder.fit_transform(train_df[object_cate_list], train_df['isFraud'])
train_df = pd.concat([train_df,temp_df],axis = 1)
train_df.drop(labels = object_cate_list,axis = 1,inplace = True )

In [20]:
temp_df = meanencoder.transform(test_df[object_cate_list])
test_df = pd.concat([test_df,temp_df],axis = 1)
test_df.drop(labels = object_cate_list,axis = 1,inplace = True )

In [21]:
del temp_df
gc.collect()

13712

In [22]:
train_df.to_csv("./Pared_train.csv",index = False)
test_df.to_csv("./Pared_test.csv",index = False)